### React Notes<br>

***

Notes made while I was coding CCArtPortfolio<br>

1Q: When importing require('dotenv').config();<br>
getting 'process not defined' error.<br>

1A: had to just use webpack.. I think it's due to react v5.0.0+ aren't supporting direct import of dotenv<br>
dotenv is only supposed to be used in the server-side node application, not at browser rendering<br>

Followed this: https://www.npmjs.com/package/dotenv-webpack<br>
create a file named: webpack.config.js<br>
and paste in the config<br>
you can now use process.env...<br>

In [ ]:
// e target value grabs the text input as it changes

import React, { useState } from 'react'
import { addDoc, collection } from 'firebase/firestore';
import { auth, db } from '../firebase/firebase.config'; 
import { useNavigate } from 'react-router-dom';

const CreatePost = () => {

  const [ title, setTitle ] = useState('');
  const [ post, setPost ] = useState('');

  return (
    <div className='container'>
      <div className='bg-light p-5 rounded mt-3'>
        <h1>Create a Post</h1>
        <div className='mb-3'>
          <label htmlFor='title' className='form-label'>Title</label>
          <input type="text" placeholder='Title' className='form-control' onChange={(e) => setTitle(e.target.value)} />
        </div>
        <div className='mb-3'>
          <label htmlFor='posts' className='form-label'>Posts</label>
          <textarea placeholder='Post...' className='form-control' onChange={(e) => setPost(e.target.value)}></textarea>
        </div>
        <button className='btn btn-dark'>Submit Post</button>
      </div>
    </div>
  )
}

export default CreatePost

In [ ]:
// complete version of createpost

import React, { useEffect, useState } from 'react'
import { addDoc, collection } from 'firebase/firestore';
import { auth, db } from '../firebase/firebase.config'; 
import { useNavigate } from 'react-router-dom';

const CreatePost = ({isAuth}) => {

  const [ title, setTitle ] = useState('');
  const [ post, setPost ] = useState('');

  let navigate = useNavigate();

  const postsCollectionRef = collection(db, 'posts');
  
  const createPost = async() => {
    if(title === '' || post === ''){
      alert("Please fill the form");
      return false;
    } else {
      try {
        await addDoc(postsCollectionRef, {
          title: title,
          post: post,
          author: {
            name: auth.currentUser.displayName,
            id: auth.currentUser.uid
          }
        })
        navigate('/')
      } catch (error) {
        console.log(error);
      }
    }
  }

  // to secure, if user is not logged in, they can't come to /createpost
  useEffect(() => {
    if(!isAuth) {
      navigate("/login");
    }
  })

  return (
    <div className='container'>
      <div className='bg-light p-5 rounded mt-3'>
        <h1>Create a Post</h1>
        <div className='mb-3'>
          <label htmlFor='title' className='form-label'>Title</label>
          <input type="text" placeholder='Title' className='form-control' onChange={(e) => setTitle(e.target.value)} />
        </div>
        <div className='mb-3'>
          <label htmlFor='posts' className='form-label'>Post</label>
          <textarea placeholder='Post...' className='form-control' onChange={(e) => setPost(e.target.value)}></textarea>
        </div>
        <button className='btn btn-dark' onClick={createPost}>Submit Post</button>
      </div>
    </div>
  )
}

export default CreatePost

In [ ]:
// bootstrap is a super fast way to add CSS
// EX:
<div className="card-body">
    <h5 className='card-title mb-3 fw-bold'>{post.title}</h5>
    <p className='card-title mb-3'>{post.post}</p>
    <p className='badge bg-dark'>{post.author.name}</p>
</div>
// you can just assign CSS properties within the className

// adding a bootstrap in index.html @react

// add this link below the index.html title
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-GLhlTQ8iRABdZLl6O3oVMWSktQOp6b7In1Zl3/Jr59b6EGGoI1aFkw7cmDA6j6gD" crossorigin="anonymous"></link>

// add this script inside the body tag
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/js/bootstrap.bundle.min.js" integrity="sha384-w76AqPfDkMBDXo30jS1Sgez6pr3x5MlQ1ZAGC+nuZB+EYdgRZgiwxhTBTkF7CXvN" crossorigin="anonymous"></script>

In [ ]:
// to get the post and delete the post
import React, { useEffect, useState } from 'react'
import { getDocs, collection, deleteDoc, doc } from 'firebase/firestore';
import { auth, db } from '../firebase/firebase.config';

const Home = () => {
  
  const [ postLists, setPostList ] = useState([]);
  const [ loading, setLoading ] = useState(false);
  const postsCollectionRef = collection(db, 'posts');

  const getPosts = async() => {
    setLoading(false);
    const data = await getDocs(postsCollectionRef);
    setPostList(data.docs.map((doc) => ({...doc.data(), id:doc.id})))
    setLoading(false);
  }

  const deletePost = async(id) => {
    const postDoc = doc(db, 'posts', id);
    await deleteDoc(postDoc);
  }

  useEffect(() => {
    getPosts();
  },[])

  return (
    <div className='homepage'>
      {postLists.length === 0 ? <h3>No post was found</h3> : postLists.map((post) => {
        return (
          <div key={post.id} className='card mb-4 shadow shadow-sm'>
            <div className='d-flex justify-content-end'>
              <button className='btn btn-danger my-3 mx-3' onClick={() => {deletePost(post.id)}}>Delete Post</button>
            </div>
            <div className="card-body">
              <h5 className='card-title mb-3 fw-bold'>{post.title}</h5>
              <p className='card-title mb-3'>{post.post}</p>
              <p className='badge bg-dark'>{post.author.name}</p>
            </div>
          </div>
        )})}
    </div>
  )
}

export default Home